In [23]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K

In [24]:
def load(paths, verbose=-1):
    '''expects images for each class in seperate dir, 
    e.g all digits in 0 class in the directory named 0 '''
    data = list()
    labels = list()
    for (i, imgpath) in enumerate(paths):
        im_gray = cv2.imread(imgpath, cv2.IMREAD_GRAYSCALE)
        image = np.array(im_gray).flatten()
        label = imgpath.split(os.path.sep)[-2]
        data.append(image/255)
        labels.append(label)
        if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
            print("[INFO] processed {}/{}".format(i + 1, len(paths)))
    return data, labels

In [5]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/trainni/trainingSample


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/trainni/trainingSample


In [6]:
img_path = '/content/drive/My Drive/trainni/trainingSample'
print(img_path)
image_paths = list(paths.list_images(img_path))
image_list, label_list = load(image_paths, verbose=10000)
lb = LabelBinarizer()
label_list = lb.fit_transform(label_list)
X_train, X_test, y_train, y_test = train_test_split(image_list, label_list, test_size=0.1, random_state=42)

/content/drive/My Drive/trainni/trainingSample


In [7]:
def create_clients(image_list, label_list, num_clients=10, initial='clients'):
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]
    data = list(zip(image_list, label_list))
    random.shuffle(data)
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]
    assert(len(shards) == len(client_names))
    return {client_names[i] : shards[i] for i in range(len(client_names))} 

In [8]:
clients = create_clients(X_train, y_train, num_clients=10, initial='client')

In [9]:

def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

In [10]:
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

In [11]:

class SimpleMLP:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        model.add(Dense(200, input_shape=(shape,)))
        model.add(Activation("relu"))
        model.add(Dense(200))
        model.add(Activation("relu"))
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

In [12]:
lr = 0.01 
comms_round = 100
loss='sparse_categorical_crossentropy'
metrics = ['accuracy']
optimizer = SGD(lr=lr, 
                decay=lr / comms_round, 
                momentum=0.9
               )                

In [13]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [14]:
smlp_global = SimpleMLP()
global_model = smlp_global.build(784, 10)
for comm_round in range(comms_round):
    global_weights = global_model.get_weights()
    scaled_local_weight_list = list()
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    for client in client_names:
        smlp_local = SimpleMLP()
        local_model = smlp_local.build(784, 10)
        local_model.compile(loss=loss, 
                      optimizer=optimizer, 
                      metrics=metrics)
        local_model.set_weights(global_weights)
        local_model.fit(clients_batched[client], epochs=1, verbose=0)
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        K.clear_session()
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    global_model.set_weights(average_weights)
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

comm_round: 0 | global_acc: 26.667% | global_loss: 2.269461154937744
comm_round: 1 | global_acc: 100.000% | global_loss: 2.185039520263672
comm_round: 2 | global_acc: 100.000% | global_loss: 2.0648200511932373
comm_round: 3 | global_acc: 100.000% | global_loss: 1.921864628791809
comm_round: 4 | global_acc: 100.000% | global_loss: 1.7937099933624268
comm_round: 5 | global_acc: 100.000% | global_loss: 1.7017714977264404
comm_round: 6 | global_acc: 100.000% | global_loss: 1.6414618492126465
comm_round: 7 | global_acc: 100.000% | global_loss: 1.6017463207244873
comm_round: 8 | global_acc: 100.000% | global_loss: 1.5745785236358643
comm_round: 9 | global_acc: 100.000% | global_loss: 1.5553886890411377
comm_round: 10 | global_acc: 100.000% | global_loss: 1.541179895401001
comm_round: 11 | global_acc: 100.000% | global_loss: 1.530368685722351
comm_round: 12 | global_acc: 100.000% | global_loss: 1.5219688415527344
comm_round: 13 | global_acc: 100.000% | global_loss: 1.5152488946914673
comm_rou

In [18]:
def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), tf.argmax(Y_test, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [22]:

SGD_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(y_train)).batch(320)
smlp_SGD = SimpleMLP()
SGD_model = smlp_SGD.build(784, 10) 
SGD_model.compile(loss=loss, 
              optimizer=optimizer, 
              metrics=metrics)
_ = SGD_model.fit(SGD_dataset, epochs=200, verbose=1)
for(X_test, Y_test) in test_batched:
        SGD_acc, SGD_loss = test_model(X_test, Y_test, SGD_model, 1)

Epoch 1/200
2/2 [==============================] - 0s 9ms/step - loss: 2.5847 - accuracy: 0.0025
Epoch 2/200
2/2 [==============================] - 0s 15ms/step - loss: 2.1169 - accuracy: 0.2958
Epoch 3/200
2/2 [==============================] - 0s 7ms/step - loss: 1.3473 - accuracy: 0.9932
Epoch 4/200
2/2 [==============================] - 0s 8ms/step - loss: 0.5901 - accuracy: 1.0000
Epoch 5/200
2/2 [==============================] - 0s 8ms/step - loss: 0.1561 - accuracy: 1.0000
Epoch 6/200
2/2 [==============================] - 0s 5ms/step - loss: 0.0349 - accuracy: 1.0000
Epoch 7/200
2/2 [==============================] - 0s 6ms/step - loss: 0.0083 - accuracy: 1.0000
Epoch 8/200
2/2 [==============================] - 0s 8ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 9/200
2/2 [==============================] - 0s 6ms/step - loss: 8.1092e-04 - accuracy: 1.0000
Epoch 10/200
2/2 [==============================] - 0s 6ms/step - loss: 4.1445e-04 - accuracy: 1.0000
Epoch 11/200
2/2 [==

In [20]:
def non_iid_x(image_list, label_list, x=1, num_intraclass_clients=10):
        non_iid_x_clients = dict()
        unique_labels = np.unique(np.array(label_list))
        random.shuffle(unique_labels)
        sub_lab_list = [unique_labels[i:i + x] for i in range(0, len(unique_labels), x)]
        for item in sub_lab_list:
            class_data = [(image, label) for (image, label) in zip(image_list, label_list) if label in item]
            images, labels = zip(*class_data)
            initial = ''
            for lab in item:
                initial = initial + lab + '_'
            intraclass_clients = create_clients(list(images), list(labels), num_intraclass_clients, initial)
            non_iid_x_clients.update(intraclass_clients)  
        return non_iid_x_clients